# RDBMS Masterclass — Hands‑On + Explanations (SQLite‑backed)
This notebook teaches the **why** and the **how** of RDBMS concepts with runnable examples.  
You’ll execute real SQL on SQLite (for universals), and you’ll see vendor‑specific notes for **Postgres/MySQL/SQL Server** where they diverge.

**What you'll learn (and *do*):**
- Mental model (parser → optimizer → executor → storage)
- Schema design, keys, constraints
- Transactions & ACID with real rollbacks
- Joins, window functions, views
- Indexing strategy & query plans (`EXPLAIN QUERY PLAN`)
- Triggers (as a stand‑in for server‑side logic)
- Performance patterns (pagination, upserts, pre‑aggregation)
- Partitioning, replication & backups (conceptual)
- Security, migrations, and an interview‑oriented summary

> ⚙️ We’ll use **SQLite** for runnable demos. Where SQLite lacks a feature, we give **Postgres/MySQL/SQL Server** examples you can paste into those engines.

## 1) Setup (run once)
We create an in‑memory SQLite DB, plus helper functions to run SQL and view results like a mini client.

In [ ]:
import sqlite3, pandas as pd, time
from caas_jupyter_tools import display_dataframe_to_user

# In-memory DB for a clean slate each run
conn = sqlite3.connect(":memory:")
conn.execute("PRAGMA foreign_keys = ON;")  # enable FK enforcement
conn.execute("PRAGMA journal_mode = WAL;") # emulate WAL (SQLite specific)
conn.execute("PRAGMA synchronous = NORMAL;")

def q(sql, params=None, many=False):
    """Run a SELECT and return a DataFrame."""
    cur = conn.cursor()
    cur.execute(sql) if not params else cur.execute(sql, params)
    cols = [d[0] for d in cur.description] if cur.description else []
    rows = cur.fetchall()
    return pd.DataFrame(rows, columns=cols)

def x(sql, params=None):
    """Execute DDL/DML and return rowcount."""
    cur = conn.cursor()
    cur.execute(sql) if not params else cur.execute(sql, params)
    conn.commit()
    return cur.rowcount

def explain(sql):
    """Return EXPLAIN QUERY PLAN for a statement."""
    return q(f"EXPLAIN QUERY PLAN {sql}")

## 2) Schema design & constraints (keys, FKs, CHECK, NOT NULL)
We’ll create a minimal **Orders** schema illustrating **PK**, **FK**, **UNIQUE**, **CHECK**, and **NOT NULL**.

In [ ]:
x("""
CREATE TABLE customers(
  id INTEGER PRIMARY KEY,        -- surrogate key (rowid-alias in SQLite)
  email TEXT NOT NULL UNIQUE,    -- natural candidate with uniqueness
  created_at TEXT NOT NULL DEFAULT (datetime('now'))
);""")

x("""
CREATE TABLE orders(
  id INTEGER PRIMARY KEY,
  customer_id INTEGER NOT NULL,
  amount NUMERIC NOT NULL CHECK (amount >= 0),
  status TEXT NOT NULL CHECK (status IN ('NEW','PAID','CANCELLED')),
  created_at TEXT NOT NULL DEFAULT (datetime('now')),
  FOREIGN KEY(customer_id) REFERENCES customers(id) ON DELETE CASCADE
);""")

# Seed a little data
x("""INSERT INTO customers(email) VALUES
 ('a@x.com'),('b@x.com'),('c@x.com');""")
x("""INSERT INTO orders(customer_id, amount, status) VALUES
 (1, 50, 'NEW'),
 (1, 20, 'PAID'),
 (2, 10, 'PAID');""")

q("SELECT * FROM customers ORDER BY id;")

**Why this matters:** Constraints encode business rules **in the database**, preventing bad data regardless of the app.  
- `UNIQUE` prevents duplicates.  
- `CHECK` enforces valid ranges/enums.  
- `FOREIGN KEY` keeps relationships consistent (and `ON DELETE CASCADE` controls delete behavior).

## 3) Transactions & ACID (Atomicity & Rollbacks)
We’ll try to insert a valid + invalid row in a **single transaction** and then **ROLLBACK** to show atomicity.

In [ ]:
# Start a transaction
conn.execute("BEGIN;")
try:
    conn.execute("INSERT INTO orders(customer_id, amount, status) VALUES (3, 5, 'NEW');")
    # Invalid status will violate CHECK and force a rollback
    conn.execute("INSERT INTO orders(customer_id, amount, status) VALUES (3, 15, 'NOT_A_STATUS');")
    conn.commit()
except Exception as e:
    conn.rollback()
    print("Rolled back due to:", e)

# Show that neither row was committed
q("SELECT * FROM orders WHERE customer_id=3;")

> **Takeaway:** Atomicity — either *all* statements succeed or **none** do.  
**Isolation levels** vary by engine (Read Committed/Repeatable Read/Serializable). SQLite’s default is serializable-like for single connection; Postgres defaults to **Read Committed**.

## 4) Joins & common patterns
We’ll add an `order_items` table and demonstrate `INNER` and `LEFT` joins.

In [ ]:
x("""
CREATE TABLE order_items(
  id INTEGER PRIMARY KEY,
  order_id INTEGER NOT NULL,
  sku TEXT NOT NULL,
  qty INTEGER NOT NULL CHECK (qty > 0),
  price NUMERIC NOT NULL CHECK (price >= 0),
  FOREIGN KEY(order_id) REFERENCES orders(id) ON DELETE CASCADE
);""")

x("""INSERT INTO order_items(order_id, sku, qty, price) VALUES
 (1, 'A', 1, 50.0),
 (2, 'B', 2, 10.0);""")

# INNER JOIN: only orders with items
q("""
SELECT o.id AS order_id, c.email, i.sku, i.qty, i.price
FROM orders o
JOIN customers c ON c.id = o.customer_id
JOIN order_items i ON i.order_id = o.id
ORDER BY o.id;
""")

**Patterns:**
- **Semi/Anti joins** (via `EXISTS` / `NOT EXISTS`) to test membership without duplicating rows.
- **LEFT JOIN** to keep unmatched rows from the left side.

In [ ]:
# Orders that have NO items (anti-join pattern)
q("""
SELECT o.id, o.status
FROM orders o
WHERE NOT EXISTS (SELECT 1 FROM order_items i WHERE i.order_id = o.id);
""")

## 5) Window functions (running totals, rankings, lag/lead)
Window functions answer “relative to other rows” questions without self-joins.

In [ ]:
# Add more orders for demos
x("""INSERT INTO orders(customer_id, amount, status, created_at) VALUES
 (1, 15, 'PAID', '2025-09-01'),
 (1, 25, 'PAID', '2025-09-02'),
 (2, 35, 'PAID', '2025-09-02');""")

q("""
SELECT customer_id,
       created_at,
       amount,
       SUM(amount) OVER (PARTITION BY customer_id ORDER BY created_at) AS running_total,
       LAG(amount) OVER (PARTITION BY customer_id ORDER BY created_at) AS prev_amount
FROM orders
ORDER BY customer_id, created_at;
""")

**Remember:** `PARTITION BY` defines peer groups, `ORDER BY` gives sequence, and optional **frames** (“rows between”) control what counts as the window.

## 6) Indexing & EXPLAIN (query plans)
We’ll index `(customer_id, created_at)` and compare plans.

In [ ]:
# Create an index compatible with our common filter
x("""CREATE INDEX idx_orders_cid_ts ON orders(customer_id, created_at);""")

# A selective query: should use the index
plan = explain("""
SELECT * FROM orders 
WHERE customer_id = 1 AND created_at >= '2025-09-01'
ORDER BY created_at;
""")
plan

**Rules of thumb:**
- Put the **most selective** column first in composite indexes.  
- Make **covering** indexes (include columns used in the query) where supported (SQLite doesn’t have INCLUDE; Postgres/SQL Server do).  
- Avoid wrapping the indexed column in functions in your predicates (or build functional indexes).

## 7) Pagination — use keyset over OFFSET for large pages
`OFFSET` grows slower with page number. Keyset pagination is **faster and consistent**.

In [ ]:
# Keyset pagination example: "fetch next rows after (last_ts, last_id)"
q("""
SELECT id, created_at, amount
FROM orders
WHERE (created_at, id) > ('2025-09-01', 0)
ORDER BY created_at, id
LIMIT 3;
""")

**Keyset pattern:** keep the last `(sort_key, id)` from the previous page and use a `WHERE` “greater than” tuple to fetch next page deterministically.

## 8) Views & pre‑aggregation
Encapsulate logic with **views**; for read‑heavy analytics, pre‑aggregate and materialize (engine‑specific).

In [ ]:
x("""
CREATE VIEW daily_revenue AS
SELECT date(created_at) AS d, SUM(amount) AS revenue
FROM orders
GROUP BY date(created_at);
""")
q("SELECT * FROM daily_revenue ORDER BY d;")

**Materialized views:**  
- **Postgres:** `CREATE MATERIALIZED VIEW ...; REFRESH MATERIALIZED VIEW ...;`  
- **SQL Server:** Indexed views (with constraints).  
- **MySQL:** Emulate with tables + events/procs.

## 9) Upserts (conflict‑aware inserts)
Syntax varies by engine. Here’s the idea:
- **Postgres**: `INSERT ... ON CONFLICT (key) DO UPDATE ...`
- **MySQL**: `INSERT ... ON DUPLICATE KEY UPDATE ...`
- **SQL Server**: `MERGE` (use carefully)
- **SQLite**: `INSERT ... ON CONFLICT(key) DO UPDATE SET ...`

In [ ]:
x("""CREATE TABLE inventory(sku TEXT PRIMARY KEY, qty INTEGER NOT NULL);""")
x("""INSERT INTO inventory(sku, qty) VALUES ('A', 10);""")
# Add +5 if exists, else insert
x("""
INSERT INTO inventory(sku, qty) VALUES ('A', 5)
ON CONFLICT(sku) DO UPDATE SET qty = qty + excluded.qty;
""")
q("SELECT * FROM inventory;")

## 10) Triggers (server‑side hooks) — use sparingly
We’ll log inventory changes. In RDBMS with stored procs (e.g., Postgres PL/pgSQL, SQL Server T‑SQL), keep business logic in code and triggers for simple invariants/audit.

In [ ]:
x("""CREATE TABLE inv_log(sku TEXT, delta INTEGER, ts TEXT DEFAULT (datetime('now')));""")

x("""
CREATE TRIGGER trg_inventory_audit
AFTER INSERT ON inventory
BEGIN
  INSERT INTO inv_log(sku, delta) VALUES (NEW.sku, NEW.qty);
END;
""")

# This INSERT will also write to inv_log
x("""INSERT INTO inventory(sku, qty) VALUES('B', 3)
ON CONFLICT(sku) DO UPDATE SET qty = qty + excluded.qty;""")

q("SELECT * FROM inv_log ORDER BY ts;")

## 11) Top‑N per group (window + filter)
Common analytics question: “top order per customer.”

In [ ]:
q("""
WITH ranked AS (
  SELECT o.*, 
         ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY amount DESC, id DESC) AS rn
  FROM orders o
)
SELECT customer_id, id AS order_id, amount
FROM ranked
WHERE rn = 1
ORDER BY customer_id;
""")

## 12) Query anti‑patterns & fixes
- `SELECT *` in hot paths → **project only needed columns**.
- Functions on indexed columns → build a **functional index** (Postgres) or store normalized columns.
- Giant `OR` chains → sometimes faster as `UNION ALL` of index‑friendly filters.
- `OFFSET` pagination on huge pages → use **keyset**.
- Stale statistics → **ANALYZE** (engine‑specific) to fix cardinality misestimates.

## 13) Normalization vs. denormalization
- **3NF/BCNF** removes update anomalies and duplicates.  
- Denormalize **only for known read paths** (summaries, pre‑joined views) and keep a source‑of‑truth table.

## 14) Partitioning (conceptual demo)
Partition large tables by **time** or **tenant** to enable **pruning** (read less data) and **retention** (drop old partitions fast).

**Postgres example:**
```sql
CREATE TABLE events (
  id bigint, ts timestamptz, payload jsonb
) PARTITION BY RANGE (ts);

CREATE TABLE events_2025_09 PARTITION OF events
  FOR VALUES FROM ('2025-09-01') TO ('2025-10-01');
```
**SQL Server:** Partition functions/schemes.  
**MySQL:** RANGE/LIST/HASH partitioning on InnoDB.

## 15) Isolation anomalies & when to raise levels
- **Read Committed**: avoids dirty reads; may see non‑repeatable/phantoms.
- **Repeatable Read** (InnoDB default): stable snapshots; may allow phantom inserts (handled via next‑key locks).
- **Serializable** (Postgres): prevents write skew with predicate locks, but expect **retries** on conflicts.
**Pattern:** keep transactions short and retry on deadlocks/serialization failures.

## 16) Security & governance (conceptual)
- Roles/Grants (least privilege), TLS in transit, TDE/OS encryption at rest.
- **Row‑Level Security** (Postgres) for multi‑tenant; column masking where supported.
- Store secrets in a vault, not in connection strings checked into code.
- Audit log: connections, DDL, high‑risk DML.

## 17) Backups, replication & recovery (conceptual)
- **Backups**: physical (fast, snapshot‑friendly) vs logical (portable dumps).  
- **WAL/Redo** enables Point‑in‑Time Recovery (PITR).  
- **Replication**: async (possible data loss) vs sync (lower RPO, higher latency).  
- Practice **restore drills** — backups are only as good as your restores.

## 18) Migrations & release engineering
Use versioned tools (Flyway/Liquibase). Prefer **backward‑compatible** steps:
1) Add nullable column
2) Backfill
3) Shift reads
4) Enforce NOT NULL/constraints
5) Drop legacy
Schedule big DDL during low traffic or use online schema change tools.

## 19) Interview crib sheet (fast answers)
- **When indexes don’t help:** low selectivity, wrapped columns, type/collation mismatch, stale stats, heavy writes.
- **Deadlocks strategy:** short txns, consistent lock order, smallest index paths, retry on specific codes.
- **Isolation choice:** default Read Committed; bump to Repeatable Read for stable reads; Serializable with retries for strict invariants.
- **Upserts:** prefer engine‑native `ON CONFLICT`/`ON DUPLICATE` over brittle MERGE if possible.
- **Pagination:** use keyset for performance/consistency.

## 20) Practice tasks (do these to lock it in)
1. Add a `payments` table with a FK to `orders`, and a `CHECK` that amount > 0. Write a query to list **unpaid** orders.
2. Create a **partial**/filtered index equivalent: in SQLite emulate by indexing a covering subset (or do it in Postgres if available).
3. Write a **windowed 7‑day rolling sum** of revenue per customer.
4. Add a **trigger** that prevents an order status from moving **PAID → NEW**.
5. Implement **keyset** pagination for `daily_revenue` view (by date).
6. Design a **partitioning** scheme (in Postgres syntax) for `orders` by month with a 6‑month retention policy.